In [ ]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 200 kB 55.6 MB/s 
     |████████████████████████████████| 199 kB 60.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b4040bcfffb9109c427670ddc7f3be5b96b67ed6b6f5881de3f69bced1bf551f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz 

!tar xf spark-3.2.2-bin-hadoop3.2.tgz

!pip install -q findspark

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

import findspark

findspark.init()

findspark.find()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField
from datetime import date, timedelta
import pyspark.sql.functions as F


spark = SparkSession.builder.appName('chandra').getOrCreate()

In [ ]:
!pwd

/content


In [ ]:
!mkdir hmc_project

In [ ]:
cd hmc_project

/content/hmc_project


In [ ]:
from google.colab import files
data = files.upload()

Saving udemy_courses.csv to udemy_courses.csv


In [ ]:
df = spark.read.csv('/content/hmc_project/udemy_courses.csv',header = True)

In [ ]:
df.show()

+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+------------------+------------------+--------------------+----------------+
|course_id|        course_title|                 url|is_paid|price|num_subscribers|num_reviews|num_lectures|             level|  content_duration| published_timestamp|         subject|
+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+------------------+------------------+--------------------+----------------+
|  1070968|Ultimate Investme...|https://www.udemy...|   True|  200|           2147|         23|          51|        All Levels|               1.5|2017-01-18T20:58:58Z|Business Finance|
|  1113822|Complete GST Cour...|https://www.udemy...|   True|   75|           2792|        923|         274|        All Levels|              39.0|2017-03-09T16:34:20Z|Business Finance|
|  1006314|Financial Modelin...|https://www.udemy...|   True|   45|        

In [ ]:
df.columns

['course_id',
 'course_title',
 'url',
 'is_paid',
 'price',
 'num_subscribers',
 'num_reviews',
 'num_lectures',
 'level',
 'content_duration',
 'published_timestamp',
 'subject']

In [ ]:
df.write.mode('overwrite').partitionBy('level').option('path', '/content/hmc_project/table').saveAsTable('Table1')

In [ ]:
df1 = spark.table('Table1')

In [ ]:
df1.show()

+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+----------------+--------------------+----------------+----------+
|course_id|        course_title|                 url|is_paid|price|num_subscribers|num_reviews|num_lectures|content_duration| published_timestamp|         subject|     level|
+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+----------------+--------------------+----------------+----------+
|  1070968|Ultimate Investme...|https://www.udemy...|   True|  200|           2147|         23|          51|             1.5|2017-01-18T20:58:58Z|Business Finance|All Levels|
|  1113822|Complete GST Cour...|https://www.udemy...|   True|   75|           2792|        923|         274|            39.0|2017-03-09T16:34:20Z|Business Finance|All Levels|
|  1210588|Beginner to Pro -...|https://www.udemy...|   True|   95|           2451|         11|          36|             3.0|

In [ ]:
df2 = spark.read.parquet('/content/hmc_project/table/*')

In [ ]:
df2.show()

+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+----------------+--------------------+----------------+
|course_id|        course_title|                 url|is_paid|price|num_subscribers|num_reviews|num_lectures|content_duration| published_timestamp|         subject|
+---------+--------------------+--------------------+-------+-----+---------------+-----------+------------+----------------+--------------------+----------------+
|  1070968|Ultimate Investme...|https://www.udemy...|   True|  200|           2147|         23|          51|             1.5|2017-01-18T20:58:58Z|Business Finance|
|  1113822|Complete GST Cour...|https://www.udemy...|   True|   75|           2792|        923|         274|            39.0|2017-03-09T16:34:20Z|Business Finance|
|  1210588|Beginner to Pro -...|https://www.udemy...|   True|   95|           2451|         11|          36|             3.0|2017-05-30T20:07:24Z|Business Finance|
|   192870|Tradi

In [ ]:
df.write.mode('overwrite').partitionBy('level').bucketBy(10,'course_id').option('path', '/content/hmc_project/table1').saveAsTable('Table1')